In [1]:
import iris
import numpy as np
from iris import quickplot as qplt
%matplotlib inline

In [2]:
# !!mkdir /usr/local/share/notebooks/data/mogreps-gg
# !s3fs mogreps-g /usr/local/share/notebooks/data/mogreps-gg -o endpoint=eu-west-2 -o public_bucket=1

In [3]:
# !!mkdir /usr/local/share/notebooks/data/mogreps-uk
# !s3fs mogreps-uk /usr/local/share/notebooks/data/mogreps-uk -o endpoint=eu-west-2 -o public_bucket=1

In [4]:
from boto.s3.connection import S3Connection
import os

os.environ['S3_USE_SIGV4'] = 'True'

def list_files(bucket, prefix='prods', n=100):
    conn = S3Connection(host='s3.eu-west-2.amazonaws.com')
    bucket = conn.get_bucket(bucket)
    results = []
    keys = iter(bucket.list(prefix=prefix))
    for i in range(n):
        try:
            results.append(next(keys))
        except StopIteration:
            pass
    return ['{}'.format(k.key) for k in results]


g_files = list_files('mogreps-g', 'prods_op_mogreps-g_20160207_00_01', 10000)
uk_files = list_files('mogreps-uk', 'prods_op_mogreps-uk_20160207_03_01', 10000)

print(len(g_files))
print(len(uk_files))

58
12


In [6]:
def get_info(fname):
    segs = fname.split('_')
    segs[-1] = segs[-1].split('.')[0]
    info = [segs[-4]] + [int(s) for s in segs[-3:]]
#     return {'run_hour': info[0], 'member': info[1], 'lead_time': info[2]}
    return info

In [7]:
import re
def get_ground_level(cube):
    '''Minimize height or maximise pressure.'''
    if any([re.match('height.*', d.name()) for d in cube.coords()]):
        for d in cube.coords():
            match = re.match('height.*', d.name())
            if match:
                name = match.group(0)
                vert_coord = cube.coord(name)
        ground_level = cube.extract(iris.Constraint(**{name: min(vert_coord.points)}))
        
    elif any([re.match('pressure.*', d.name()) for d in cube.coords()]):
        for d in cube.coords():
            match = re.match('pressure.*', d.name())
            if match:
                name = match.group(0)
                vert_coord = cube.coord(name)
        ground_level = cube.extract(iris.Constraint(**{name: max(vert_coord.points)}))
    
    else:
        ground_level = cube
        name = None
    
    return (ground_level, name)

In [8]:
def unrotate(uk_cube, g_cube):
    glat = uk_cube.coord('grid_latitude').points
    glon = uk_cube.coord('grid_longitude').points
    x, y = np.meshgrid(glon, glat)
    cs = uk_cube.coord_system()
    lons, lats = iris.analysis.cartography.unrotate_pole(x, y, cs.grid_north_pole_longitude, cs.grid_north_pole_latitude)
    clons = iris.coords.DimCoord(lons[0,:], standard_name='longitude', var_name='longitude', units=g_cube.coord('longitude').units, coord_system=iris.coord_systems.GeogCS(6371229.0))
    clats = iris.coords.DimCoord(lats[:,0], standard_name='latitude', var_name='latitude', units=g_cube.coord('latitude').units, coord_system=iris.coord_systems.GeogCS(6371229.0))

    uk_unrotate = uk_cube.copy()
    uk_unrotate.remove_coord('grid_latitude')
    uk_unrotate.add_dim_coord(clats, 0)
    uk_unrotate.remove_coord('grid_longitude')
    uk_unrotate.add_dim_coord(clons, 1)
    return uk_unrotate

In [9]:
def get_uk_from_global(g_cube, uk_cube):
    minlat = uk_cube.coord('latitude').points[0]
    maxlat = uk_cube.coord('latitude').points[-1]
    minlon = uk_cube.coord('longitude').points[0]
    maxlon = uk_cube.coord('longitude').points[-1]
    
    g_cube_uk = g_cube.intersection(latitude=(minlat,maxlat)).intersection(longitude=(minlon,maxlon))
    return g_cube_uk.regrid(grid=uk_cube, scheme=iris.analysis.Linear())

In [10]:
def get_diff(uk_cube, g_cube):   
    diff = g_cube - uk_cube
    diff.add_aux_coord(uk_cube.coord('time'))
    diff.coord('time').var_name = 'time'
    return diff

In [11]:
def get_diffs(param):
    diffs = iris.cube.CubeList([])
    uks = iris.cube.CubeList([])
    gs = iris.cube.CubeList([])
    
    for fname in uk_files:
        info = get_info(fname)
        uk_fname = fname
        g_fname = 'prods_op_mogreps-g_{}_{:02d}_{:02d}_{:03d}.nc'.format(info[0], info[1]-3, info[2], info[3]+3)

        if g_fname in g_files:
            try:
                g_cubes = iris.load('./data/mogreps-gg/' + g_fname)
                uk_cubes = iris.load('./data/mogreps-uk/' + uk_fname)
                uk_cube = uk_cubes[[c.name() for c in uk_cubes].index(param)]
                g_cube = g_cubes[[c.name() for c in g_cubes].index(param)]
                
                uk_cube = uk_cube.extract(iris.Constraint(time=uk_cube.coord('time').points[-1]))
                g_cube = g_cube.extract(iris.Constraint(time=uk_cube.coord('time').points[-1]))
                print(uk_cube.coord('time'))
                
                uk_cube, uk_h = get_ground_level(uk_cube)
                if uk_h: uk_cube.remove_coord(uk_h)
                g_cube, g_h = get_ground_level(g_cube)
                if g_h: g_cube.remove_coord(g_h)
                    
                uk_unrotate = unrotate(uk_cube, g_cube)
                g_cube_uk = get_uk_from_global(g_cube, uk_unrotate)

                g_regrid = g_cube_uk.regrid(grid=uk_unrotate, scheme=iris.analysis.Nearest())
                diff_hr = get_diff(uk_unrotate, g_regrid)
                diffs.append(diff_hr)
                uks.append(uk_unrotate)
                gs.append(g_regrid)
            except (IndexError, AttributeError):
                pass
    return (diffs, uks, gs)

In [12]:
variable = 'low_type_cloud_area_fraction'

In [ ]:
%%time 
diffs, uks, gs = get_diffs(variable)

/opt/conda/lib/python3.5/site-packages/iris/fileformats/cf.py:1140: IrisDeprecation: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warn_deprecated(msg)


DimCoord([2016-02-07 06:00:00], standard_name='time', calendar='gregorian', var_name='time_2')
DimCoord([2016-02-07 09:00:00], standard_name='time', calendar='gregorian', var_name='time_0')
DimCoord([2016-02-07 12:00:00], standard_name='time', calendar='gregorian', var_name='time_0')
DimCoord([2016-02-07 15:00:00], standard_name='time', calendar='gregorian', var_name='time_0')
DimCoord([2016-02-07 18:00:00], standard_name='time', calendar='gregorian', var_name='time_0')
DimCoord([2016-02-07 21:00:00], standard_name='time', calendar='gregorian', var_name='time_0')
DimCoord([2016-02-08 00:00:00], standard_name='time', calendar='gregorian', var_name='time_0')


In [ ]:
qplt.pcolormesh(uks[0], cmap='viridis')
qplt.show()
qplt.pcolormesh(gs[0], cmap='viridis')
qplt.show()
qplt.pcolormesh(diffs[0], cmap='viridis')

In [ ]:
[pd.collapsed(['latitude', 'longitude'], iris.analysis.RMS).data.item(0) for pd in diffs]

In [ ]:
diffs_cube = diffs.merge()[0]
print(diffs_cube)

In [ ]:
abs_diffs = [iris.analysis.maths.abs(c) for c in diffs]

In [ ]:
[pd.collapsed(['latitude', 'longitude'], iris.analysis.RMS).data.item(0) for pd in abs_diffs]

In [ ]:
for i in range(11):
    qplt.pcolormesh(diffs[i], cmap='viridis')
    qplt.show()